# 과제 2-B





In [1]:
!pip install jupyter_black
!pip install sacremoses
!pip install datasets

In [2]:
import jupyter_black

jupyter_black.load()

In [3]:
backend = "mps"

## Last word prediction dataset 준비
    
  - ✅ 기존의 IMDB dataset을 그대로 활용합니다.
  - ✅ `collate_fn` 함수에 다음 수정사항들을 반영하면 됩니다.
      - ✅ Label은 text를 token으로 변환했을 때 마지막 token의 id로 설정합니다.
      - ✅ 입력 data는 마지막 token을 제외한 나머지 token들의 list로 설정합니다.
  - ✅ `from torch.nn.utils.rnn import pad_sequence`를 import해서 사용하셔도 좋습니다.
  - ✅ Truncation은 기존과 똑같이 진행하시면 됩니다.

In [4]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


def collate_fn(batch):
    max_len = 400
    inputs, labels = [], []
    for row in batch:
        tokens = tokenizer(row["text"], truncation=True, max_length=max_len).input_ids

        inputs.append(tokens[:-1])
        labels.append(tokens[-1])

    inputs = pad_sequence(
        [torch.tensor(input_ids) for input_ids in inputs],
        batch_first=True,
        padding_value=tokenizer.pad_token_id,
    )
    labels = torch.LongTensor(labels)

    return inputs, labels


train_loader = DataLoader(
    ds["train"], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds["test"], batch_size=64, shuffle=False, collate_fn=collate_fn
)

/Users/bh/workspaces/studies/hh-ai/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Self-attention

In [5]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
    def __init__(self, input_dim, d_model):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        q, k, v = self.wq(x), self.wk(x), self.wv(x)
        score = torch.matmul(
            q, k.transpose(-1, -2)
        )  # (B, S, D) * (B, D, S) = (B, S, S)
        score = score / sqrt(self.d_model)

        if mask is not None:
            score = score + (mask * -1e9)

        score = self.softmax(score)
        result = torch.matmul(score, v)
        result = self.dense(result)

        return result

In [6]:
from torch import nn
from math import sqrt


class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, d_model, n_heads):
        super().__init__()

        assert d_model % n_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.n_heads = n_heads

        self.depth = d_model // n_heads

        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def split_heads(self, x):
        batch_size, seq_len, _ = x.size()
        # S, D -> S, H, D' -> H, S, D'
        return x.view(batch_size, seq_len, self.n_heads, self.depth).transpose(1, 2)

    def forward(self, x, mask):
        batch_size = x.size(0)

        # H, S, D'
        q, k, v = (
            self.split_heads(self.wq(x)),
            self.split_heads(self.wk(x)),
            self.split_heads(self.wv(x)),
        )

        # (H, S, D') * (H, D', S) -> H, S, S
        score = torch.matmul(q, k.transpose(-1, -2)) / sqrt(self.depth)

        if mask is not None:
            mask = mask.unsqueeze(1)
            score = score + (mask * -1e9)

        score = self.softmax(score)
        result = torch.matmul(score, v)  # (H, S, S) * (H, S, D') -> H, S, D'

        # H, S, D' -> S, H, D' -> S, D
        result = result.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        result = self.dense(result)

        return result

In [7]:
class TransformerLayer(nn.Module):
    def __init__(self, input_dim, d_model, dff, n_heads, dropout_rate=0.3):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff

        self.mha = MultiHeadAttention(input_dim, d_model, n_heads)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff), nn.ReLU(), nn.Linear(dff, d_model)
        )

        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, mask):
        x1 = self.mha(x, mask)
        x1 = self.dropout(x1)
        x1 = self.layernorm1(x1 + x)

        x2 = self.ffn(x1)
        x2 = self.dropout(x2)
        x2 = self.layernorm2(x2 + x1)

        return x2

## Positional encoding

In [8]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, None], np.arange(d_model)[None, :], d_model
    )
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


- Loss function 및 classifier output 변경
    - ✅ 마지막 token id를 예측하는 것이기 때문에 binary classification이 아닌 일반적인 classification 문제로 바뀝니다. MNIST 과제에서 했던 것 처럼 `nn.CrossEntropy` loss와 `TextClassifier`의 출력 차원을 잘 조정하여 task를 풀 수 있도록 수정하시면 됩니다.

In [9]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, dff, n_heads):
        super().__init__()

        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.dff = dff

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.parameter.Parameter(
            positional_encoding(max_len, d_model), requires_grad=False
        )
        self.layers = nn.ModuleList(
            [TransformerLayer(d_model, d_model, dff, n_heads) for _ in range(n_layers)]
        )
        self.classification = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        mask = x == tokenizer.pad_token_id
        mask = mask[:, None, :]
        seq_len = x.shape[1]

        x = self.embedding(x)
        x = x * sqrt(self.d_model)
        x = x + self.pos_encoding[:, :seq_len]

        for layer in self.layers:
            x = layer(x, mask)

        x = x[:, -1]
        x = self.classification(x)

        return x


model = TextClassifier(len(tokenizer), 32, 5, 128, 4)

## 학습


In [10]:
from torch.optim import Adam

lr = 0.001
model = model.to(backend)
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [11]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
    cnt = 0
    acc = 0

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(backend), labels.to(backend)

        preds = model(inputs)
        preds = torch.argmax(preds, dim=-1)
        # preds = (preds > 0).long()[..., 0]

        cnt += labels.shape[0]
        acc += (labels == preds).sum().item()

    return acc / cnt

In [12]:
%%time

# 시간관계상 50 -> 3으로 변경
n_epochs = 3

for epoch in range(n_epochs):
    total_loss = 0.0
    model.train()

    for data in train_loader:
        model.zero_grad()
        inputs, labels = data
        inputs, labels = inputs.to(backend), labels.to(backend)

        preds = model(inputs)

        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

    with torch.no_grad():
        model.eval()
        train_acc = accuracy(model, train_loader)
        test_acc = accuracy(model, test_loader)
        print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 434.22149012889713
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   1 | Train Loss: 2.3325765430927277
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   2 | Train Loss: 0.8341400201898068
=========> Train acc: 1.000 | Test acc: 1.000
CPU times: user 5min 39s, sys: 24.2 s, total: 6min 4s
Wall time: 22min 49s
